In [1]:
!pip install --upgrade gdown

  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.3
    Uninstalling gdown-4.7.3:
      Successfully uninstalled gdown-4.7.3


In [2]:
##############################
# google drive mount
##############################
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
##############################
#  download and extract image data
##############################

import functools
import os
import gdown
DATASET_ZIP = '/content/drive/MyDrive/IAB/images.zip'
if not os.path.exists('/content/kfood') and os.path.exists(DATASET_ZIP):
    os.remove(DATASET_ZIP)
gdown.cached_download(
        'https://drive.google.com/uc?id=1Ft4xsxTmFjLX-NAB9CTD7QARYm3-ZyeB',
        path=DATASET_ZIP,
        md5='daa6095ed41e47576d9aa995bcf3c938',
        postprocess=functools.partial(gdown.extractall, to='/content/'),
)

/usr/local/lib/python3.10/dist-packages/gdown/cached_download.py:102: FutureWarning: md5 is deprecated in favor of hash. Please use hash='md5:xxx...' instead.
  warnings.warn(
Cached downloading...
Hash: md5:daa6095ed41e47576d9aa995bcf3c938
From (original): https://drive.google.com/uc?id=1Ft4xsxTmFjLX-NAB9CTD7QARYm3-ZyeB
From (redirected): https://drive.google.com/uc?id=1Ft4xsxTmFjLX-NAB9CTD7QARYm3-ZyeB&confirm=t&uuid=05e628ee-6ba5-4ce1-8d43-359629c9ed03
To: /content/drive/MyDrive/IAB/images.zip
100%|██████████| 119M/119M [00:01<00:00, 76.8MB/s]


'/content/drive/MyDrive/IAB/images.zip'

In [5]:
import torch # pytorch
import torch.nn.functional as F # loss function
from tqdm import tqdm # progress bar
import argparse # command line argument parser
import os # file path
import random # random seed
import numpy as np # numpy
from PIL import Image # image file
from torchvision import transforms # image transform
from torch.utils.data import Dataset, DataLoader # dataset, dataloader
import torch.nn as nn # neural network
import pandas as pd # dataframe
import easydict # making easy dictionary for argument parser

In [6]:
##############################
# dataset class definition
##############################
class KfoodDataset(Dataset):
    def __init__(self, split):
        self.split=split
        with open('/content/drive/MyDrive/IAB/{}.csv'.format(self.split)) as f:
            df = pd.read_csv(f)
        self.ids = df['id'].tolist()
        if self.split == 'test':
            self.categories = [0] * len(self.ids)
        else:
            self.categories = df['category'].tolist()
        assert(len(self.ids) == len(self.categories)), "Data error"

        if self.split == 'train':
            self.transforms=transforms.Compose([
                        transforms.Resize(255),
                        transforms.RandomCrop(224),
                        transforms.ToTensor(),
                        transforms.Normalize((0.485, 0.456, 0.406),
                                             (0.229, 0.224, 0.225))])
        else:
            self.transforms=transforms.Compose([
                        transforms.Resize(255),
                        transforms.CenterCrop(224),
                        transforms.ToTensor(),
                        transforms.Normalize((0.485, 0.456, 0.406),
                                             (0.229, 0.224, 0.225))])
    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        im_id = self.ids[idx]
        label = int(self.categories[idx])

        # Open image
        img = Image.open('/content/kfood/{}/{}'.format(self.split,im_id))
        img = img.convert("RGB")
        return self.transforms(img), label

##############################
# data loader
##############################
def get_dataloader(split, batch_size, shuffle, num_workders):
    dataset = KfoodDataset(split)
    dataloader = DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=shuffle,
            num_workers=num_workders
    )
    return dataloader

In [7]:
##############################
# Simple model class definiton
##############################
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        layers = []
        in_channels = 3
        for v in [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512]:
            if v == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
                layers += [conv2d, nn.ReLU(inplace=True)]
                in_channels = v
        self.cnn_layers = nn.Sequential(*layers)
        self.pooling = nn.AdaptiveAvgPool2d((1,1))
        self.classifier = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(512, 5)
        )

    def forward(self, x):
        x = self.cnn_layers(x)
        x = self.pooling(x)
        x = x.squeeze(2).squeeze(2)
        x = self.classifier(x)
        return x

In [8]:
##############################
# Main model class definiton
##############################
class Jumper(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Jumper, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)

        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.batch_norm3 = nn.BatchNorm2d(out_channels*self.expansion)

        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.ReLU()

    def forward(self, x):
        identity = x.clone()
        x = self.relu(self.batch_norm1(self.conv1(x)))
        x = self.relu(self.batch_norm2(self.conv2(x)))
        x = self.conv3(x)
        x = self.batch_norm3(x)
        if self.i_downsample is not None:
            identity = self.i_downsample(identity)
        x+=identity
        x=self.relu(x)

        return x

class TrueModel(nn.Module):
    def __init__(self, num_labels, num_channels=3):
        super(TrueModel, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size = 3, stride=2, padding=1)

        self.Jumper = Jumper
        self.layer1 = self._make_layer(Jumper, 3, planes=64)
        self.layer2 = self._make_layer(Jumper, 8, planes=128, stride=2)
        self.layer3 = self._make_layer(Jumper, 36, planes=256, stride=2)
        self.layer4 = self._make_layer(Jumper, 3, planes=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*Jumper.expansion, num_labels)

    def forward(self, x):
        x = self.relu(self.batch_norm1(self.conv1(x)))
        x = self.max_pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x

    def _make_layer(self, Jumper, blocks, planes, stride=1):
        ii_downsample = None
        layers = []

        if stride != 1 or self.in_channels != planes*Jumper.expansion:
            ii_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, planes*Jumper.expansion, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes*Jumper.expansion)
            )

        layers.append(Jumper(self.in_channels, planes, i_downsample=ii_downsample, stride=stride))
        self.in_channels = planes*Jumper.expansion

        for i in range(blocks-1):
            layers.append(Jumper(self.in_channels, planes))

        return nn.Sequential(*layers)

ResNet_model = TrueModel(num_labels=len(os.listdir('/content/kfood/train')))
# model = torch.compile(model)
print(ResNet_model)

TrueModel(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (max_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Jumper(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (batch_norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (batch_norm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (i_downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, mo

In [13]:
##############################
# validation
##############################
def val(args, val_dataloader, model):
    model.eval()
    result = []
    for imgs, label in tqdm(val_dataloader, ncols=80):
        imgs = imgs.cuda()
        with torch.no_grad():
            pred = model(imgs)
            pred_id = torch.argmax(pred, 1).cpu()
        result += (label == pred_id).long().tolist()
    print("Acc: ", sum(result) / len(result))

##############################
# train
##############################
def train(args, train_dataloader, val_dataloader,
          model, optimizer, start_epoch):
    print('Start to try ', args.max_epoch, ' epoch learning from ', start_epoch, '!')
    for epoch in range(args.max_epoch):
        model.train()
        tr_loss, num_tr_steps = 0, 0
        for imgs, label in tqdm(train_dataloader, ncols=80):
            imgs = imgs.cuda()
            label = label.cuda()
            pred = model(imgs)
            loss = F.cross_entropy(pred, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            tr_loss += loss.cpu().item()
            num_tr_steps += 1

        print('===== Epoch %d done.' % (epoch+1))
        print('===== Average training loss', tr_loss / num_tr_steps)

        # Evaluation
        val(args, val_dataloader, model)

        # # Save model
        # torch.save({'epoch': epoch + 1,
        #             'model_state': model.state_dict(),
        #             'optimizer_state' : optimizer.state_dict()},
        #             os.path.join(args.checkpoints_dir, 'last_ckpt.pth'))
        # Save model
        torch.save({'epoch': epoch + 1 + start_epoch, # note) 기존에 제가 start_epoch를 더하지 않아, 추가로 학습을 진행할 시 epoch이 무지막지하게 늘어나는 버그가 있습니다. 쉽게 수정 가능하나 시간 문제로 fix는 못 했습니다.
                    'model_state': model.state_dict(),
                    'optimizer_state' : optimizer.state_dict()},
                    os.path.join(args.checkpoints_dir, 'last_ckpt.pth'))

In [14]:
##############################
# main function for training
##############################
def set_train(args):
    ###############################################
    # WARNING: DO NOT CHANGE THIS PART (BEGINNING)
    ###############################################
    seed = 2023
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    ###############################################
    # WARNING: DO NOT CHANGE THIS PART (END)
    ###############################################

    # Make logdir
    if not os.path.exists(args.checkpoints_dir):
        os.makedirs(args.checkpoints_dir)

    # Load dataset
    train_dataloader = get_dataloader('train', args.bs, True, args.nw)
    val_dataloader = get_dataloader('val', args.bs, False, args.nw)

    # Model
    model = ResNet_model

    optimizer = torch.optim.SGD(
        model.parameters(), lr=args.lr, momentum=0.9,
        weight_decay=args.wd)

    # If you want to use Adam optimizer, use below code
    # optimizer = torch.optim.Adam(
    #     model.parameters(), lr=args.lr, eps=1e-08,
    #     weight_decay=args.wd, amsgrad=False)

    model.cuda()
    if args.tuning:
        ckpt = torch.load(os.path.join(args.checkpoints_dir, 'last_ckpt.pth'))
        model.load_state_dict(ckpt['model_state'])
        optimizer.load_state_dict(ckpt['optimizer_state'])
        start_epoch = ckpt['epoch']
    else:
        start_epoch = 0

    train(args, train_dataloader, val_dataloader,
          model, optimizer, start_epoch)

In [15]:
##############################
# arguments setting for trainig
##############################
args = easydict.EasyDict({
     "bs": 8,            # Batch size
     "nw": 2,            # Number of workers for dataloader
     "lr": 0.01,         # Learning rate
     "wd": 0.0001,       # Weight decay
     "max_epoch": 1,     # Number of epochs
     "checkpoints_dir": "/content/drive/MyDrive/IAB/checkpoints",
     "tuning": False      # If True, continue training from last checkpoint
})

set_train(args)

Start to try  1  epoch learning from  0 !


100%|█████████████████████████████████████████| 250/250 [00:59<00:00,  4.22it/s]


===== Epoch 1 done.
===== Average training loss 2.258508573055267


100%|███████████████████████████████████████████| 63/63 [00:06<00:00, 10.48it/s]


Acc:  0.354


In [17]:
##############################
# test
##############################
def test(args, test_dataloader, model):
    model.eval()
    result = []
    for imgs, label in tqdm(test_dataloader, ncols=80):
        imgs = imgs.cuda()
        with torch.no_grad():
            pred = model(imgs)
            pred_id = torch.argmax(pred, 1).cpu()
        result += pred_id.tolist()
    return result

##############################
# main function for test
##############################
def main_test(args):
    # Load dataset
    test_dataloader = get_dataloader('test', args.bs, False, args.nw)

    # Model
    # model = SimpleModel()
    model = ResNet_model
    model.cuda()
    ckpt = torch.load(os.path.join(args.checkpoints_dir, 'last_ckpt.pth'))
    model.load_state_dict(ckpt['model_state'])

    result = test(args, test_dataloader, model)

    # Make csv file
    df = pd.DataFrame({'id': test_dataloader.dataset.ids,
                       'category': result})
    df.to_csv(args.output_csv_file, index=False)

In [18]:
# Do prediction for submit

##############################
# arguments setting for test
##############################
args = easydict.EasyDict({
     "bs": 16,            # Batch size
     "nw": 2,            # Number of workers for dataloader
     "checkpoints_dir": "/content/drive/MyDrive/IAB/checkpoints",
     "output_csv_file": "/content/drive/MyDrive/IAB/out.csv"
})

main_test(args)

100%|███████████████████████████████████████████| 63/63 [00:09<00:00,  6.90it/s]
